In [45]:
import pandas as pd
import traceback
import os,sys

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from MetaCreate import MetaCreate
from AudioManipulator import AudioManipulator
from preprocess_uitls import pad_audio_files, featurize
from utils import char_map,index_map, int_sequence_to_text ,text_to_int_sequence


import librosa
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

    
# from keras.utils.vis_utils import plot_model


import _pickle as pickle
from numpy.lib.stride_tricks import as_strided

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
# from keras.callbacks import ModelCheckpoint   

In [3]:
import mlflow
import mlflow.tensorflow

In [25]:
data=pd.read_json('../artifacts/meta.json')

In [26]:
data.text.count()

10179

In [27]:
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)

In [28]:
data.text.count()

9357

In [29]:
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [30]:
print(len(features),len(data.text.to_numpy()))

9357 9357


In [32]:
valid_data=pd.read_json("../artifacts/valid_meta.json")

In [33]:
valid_data.text.count()

1990

In [34]:
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)

In [35]:
valid_data.text.count()

1990

In [36]:
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

In [37]:
print(len(valid_features),len(valid_data.text.to_numpy()))

1990 1990


In [38]:
from Batch import Batch

In [40]:

MODEL_NAME = "bidirectional_rnn_gpu"
EPOCHS=50
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

# Bidirectional RNN

In [41]:
bidirectional_rnn_gpu=bidirectional_rnn_model_gpu(input_dim=26, 
                        units=250)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 500)         417000    
_________________________________________________________________
norm_bidir_rnn (BatchNormali (None, None, 500)         2000      
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 29)          14529     
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 433,529
Trainable params: 432,529
Non-trainable params: 1,000
_________________________________________________________________
None


In [42]:
from Train import train

In [24]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Bidirectional RNN GPU")
train(audio_gen, input_to_softmax=bidirectional_rnn_gpu, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

2021/08/10 08:06:27 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c06cf7e2747a45a98030c12a3b0e0f41', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
37/37 [==============================] - 51s 824ms/step - loss: 183.6048 - val_loss: 229.5184
Epoch 2/50
37/37 [==============================] - 28s 750ms/step - loss: 139.3651 - val_loss: 234.0773
Epoch 3/50
37/37 [==============================] - 27s 745ms/step - loss: 134.1298 - val_loss: 206.2278
Epoch 4/50
37/37 [==============================] - 28s 757ms/step - loss: 130.5624 - val_loss: 170.3765
Epoch 5/50
37/37 [==============================] - 28s 750ms/step - loss: 127.9668 - val_loss: 156.5076
Epoch 6/50
37/37 [==============================] - 27s 746ms/step - loss: 125.6403 - val_loss: 140.2581
Epoch 7/50
37/37 [==============================] - 28s 763ms/step - loss: 124.9986 - val_loss: 147.2280
Epoch 8/50
37/37 [==============================] - 28s 747ms/step - loss: 123.7347 - val_loss: 138.5409
Epoch 9/50
37/37 [==============================] - 28s 746ms/step - loss: 122.8607 - val_loss: 136.5978
Epoch 10/50
37/37 [==============================] - 27

2021/08/10 08:30:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [62]:

MODEL_NAME = "simple_rnn"
EPOCHS=10
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

# Simple RNN

In [39]:
simple_rnn_model = simple_rnn(input_dim=26,
                units=5,
                activation='tanh',
                output_dim=29)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
rnn (GRU)                    (None, None, 5)           495       
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 5)           20        
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 29)          174       
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 689
Trainable params: 679
Non-trainable params: 10
_________________________________________________________________
None


In [68]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Simple RNN")
train(audio_gen, input_to_softmax=simple_rnn_model, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/09 14:35:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '822ca4f8a42f49e6a9915585f0ca1177', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
37/37 [==============================] - 28s 661ms/step - loss: 207.9806 - val_loss: 223.8220
Epoch 2/10
37/37 [==============================] - 25s 672ms/step - loss: 172.3360 - val_loss: 226.4976
Epoch 3/10
37/37 [==============================] - 34s 916ms/step - loss: 172.2853 - val_loss: 199.5817
Epoch 4/10
37/37 [==============================] - 25s 677ms/step - loss: 171.7939 - val_loss: 196.6637
Epoch 5/10
37/37 [==============================] - 23s 629ms/step - loss: 172.3429 - val_loss: 198.5662
Epoch 6/10
37/37 [==============================] - 24s 642ms/step - loss: 172.3947 - val_loss: 180.0471
Epoch 7/10
37/37 [==============================] - 23s 632ms/step - loss: 172.1374 - val_loss: 174.4812
Epoch 8/10
37/37 [==============================] - 26s 695ms/step - loss: 171.5404 - val_loss: 174.9494
Epoch 9/10
37/37 [==============================] - 22s 593ms/step - loss: 171.3645 - val_loss: 161.4830
Epoch 10/10
37/37 [==============================] - 25

2021/08/09 14:39:31 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


# CNN RNN

In [46]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0   

In [47]:

MODEL_NAME = "cnn_rnn"
EPOCHS=50
MINI_BATCH_SIZE = 200
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [48]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("CNN RNN")
train(audio_gen, input_to_softmax=cnn_rnn, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/10 10:36:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '022c17e977e64e369982a259aa024365', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
46/46 [==============================] - 102s 870ms/step - loss: nan - val_loss: nan
Epoch 2/50
46/46 [==============================] - 34s 747ms/step - loss: nan - val_loss: nan
Epoch 3/50
25/46 [===============>..............] - ETA: 16s - loss: nan

# Deep Speech

In [18]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.2,
                number_of_layers=2,
                output_dim=len(char_map)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 250)         264000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 250)         1000      
_________________________________________________________________
final_layer_of_rnn (GRU)     (None, None, 250)         376500    
_________________________________________________________________
bt_rnn_final (BatchNormaliza (None, None, 250)         1000

In [19]:

MODEL_NAME = "deep_speech"
EPOCHS=200
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [20]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech")
train(audio_gen, input_to_softmax=deep_speech, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/10 01:31:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '358e2b61010343d8afcb58844fc80bc2', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/200
37/37 [==============================] - 68s 1s/step - loss: 221.2112 - val_loss: 226.1199
Epoch 2/200
37/37 [==============================] - 32s 869ms/step - loss: 167.7959 - val_loss: 170.9646
Epoch 3/200
37/37 [==============================] - 36s 964ms/step - loss: 137.1097 - val_loss: 133.5363
Epoch 4/200
37/37 [==============================] - 31s 847ms/step - loss: 117.7491 - val_loss: 128.7209
Epoch 5/200
37/37 [==============================] - 35s 965ms/step - loss: 108.3966 - val_loss: 111.8907
Epoch 6/200
37/37 [==============================] - 32s 863ms/step - loss: 101.9971 - val_loss: 107.1758
Epoch 7/200
37/37 [==============================] - 33s 882ms/step - loss: 97.3106 - val_loss: 99.3448
Epoch 8/200
37/37 [==============================] - 34s 922ms/step - loss: 93.6979 - val_loss: 95.0559
Epoch 9/200
37/37 [==============================] - 32s 877ms/step - loss: 90.7185 - val_loss: 93.2330
Epoch 10/200
37/37 [==============================] - 3

KeyboardInterrupt: 

# WER

In [62]:
with open('models/simple_rnn_model.pickle', 'rb') as f:
        simple_rnn_hist=pickle.load( f)

In [66]:
# simple_rnn_model.load_weights(simple_rnn_hist)

In [25]:
import importlib

In [119]:
import wer

In [32]:
import Train
importlib.reload(Train)
from Train import  train

In [27]:
import utils

In [28]:
importlib.reload(utils)

<module 'utils' from 'd:\\same\\Documents\\10Acadamy\\Week 4\\System\\Week-4\\scripts\\utils.py'>

In [197]:
importlib.reload(wer)

<module 'wer' from 'd:\\same\\Documents\\10Acadamy\\Week 4\\System\\Week-4\\scripts\\wer.py'>

In [198]:
from utils import  char_map,index_map,int_sequence_to_text,text_to_int_sequence

In [132]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [133]:
index_map

{0: '',
 1: '<SPACE>',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'q',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'x',
 26: 'y',
 27: 'z'}

In [218]:
text_to_int_sequence("z")

reliad


[27]

In [216]:
int_sequence_to_text([28, 0])

['z']

In [199]:
from wer import predict

In [ ]:
predict(audio_gen,17,'validation',deep_speech)

Error: Kernel is dead

In [158]:
calculate_wer(cnn_rnn,"CNN RNN",audio_gen, 'validation',1000)


processed 100 in 0 minutes
processed 200 in 0 minutes
processed 300 in 1 minutes
processed 400 in 1 minutes
processed 500 in 1 minutes
processed 600 in 2 minutes
processed 700 in 2 minutes
processed 800 in 3 minutes
processed 900 in 3 minutes
Total time: 3.843780 minutes


[65,
 44,
 85,
 61,
 70,
 50,
 42,
 71,
 64,
 44,
 35,
 26,
 44,
 43,
 34,
 37,
 28,
 82,
 51,
 38,
 37,
 74,
 42,
 33,
 49,
 57,
 40,
 71,
 58,
 58,
 76,
 52,
 42,
 54,
 74,
 60,
 61,
 54,
 53,
 45,
 49,
 70,
 46,
 42,
 50,
 45,
 40,
 66,
 51,
 47,
 26,
 80,
 68,
 39,
 68,
 46,
 56,
 45,
 43,
 61,
 42,
 41,
 46,
 45,
 65,
 77,
 44,
 92,
 101,
 35,
 44,
 39,
 36,
 82,
 51,
 50,
 64,
 58,
 53,
 49,
 86,
 43,
 73,
 59,
 59,
 26,
 63,
 67,
 31,
 25,
 37,
 49,
 50,
 36,
 51,
 41,
 52,
 32,
 48,
 46,
 32,
 87,
 60,
 25,
 66,
 41,
 40,
 47,
 60,
 48,
 79,
 46,
 104,
 44,
 38,
 69,
 72,
 90,
 71,
 45,
 37,
 31,
 57,
 42,
 46,
 27,
 42,
 38,
 54,
 40,
 43,
 38,
 28,
 42,
 40,
 52,
 50,
 78,
 60,
 42,
 39,
 42,
 28,
 25,
 25,
 41,
 29,
 30,
 58,
 37,
 23,
 41,
 78,
 45,
 113,
 58,
 60,
 37,
 33,
 31,
 71,
 37,
 32,
 32,
 65,
 56,
 51,
 83,
 24,
 35,
 47,
 65,
 26,
 25,
 48,
 36,
 38,
 57,
 46,
 55,
 35,
 41,
 42,
 65,
 62,
 49,
 37,
 49,
 39,
 27,
 51,
 71,
 42,
 51,
 37,
 68,
 48,
 43,
 39,
 5